In [1]:
import pandas as pd
import numpy as np

# Read taxonomy table

In [2]:
df_tax = pd.read_csv('taxonomy.tsv', sep='\t')
taxonomy = []
for idx in df_tax.index:
    taxon = df_tax.loc[idx,'Taxon']
    taxon = taxon.split(';')
    res = [df_tax.loc[idx,'Feature ID']]
    lowest_classified_taxon = 'unclassified'
    for i in np.arange(7):
        if i < len(taxon):
            res.append(taxon[i])
            if 'unclassified' not in taxon[i] and 'uncultured' not in taxon[i] and 'unidentified' not in taxon[i]:
                lowest_classified_taxon = taxon[i]
        else:
            res.append('unclassified')
    res.append(lowest_classified_taxon)
    taxonomy.append(res)
    
df_tax = pd.merge(df_tax, pd.DataFrame(taxonomy, columns=['Feature ID','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']), left_on='Feature ID', right_on='Feature ID', how='left')
df_tax = df_tax[['Feature ID','Confidence','Kingdom','Phylum','Class','Order','Family','Genus','Species','LowestClassifiedTaxon']]
df_tax = df_tax.rename({'Feature ID':'ASV'}, axis=1).set_index('ASV')

# remove genus tag in species
df_tax['Species'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['Species'])]
#df_tax['LowestTaxon'] = [y.replace(x+'-','') for x,y in zip(df_tax['Genus'], df_tax['LowestTaxon'])]

# # select only bacteria
# df_tax = df_tax[df_tax.Kingdom=='Bacteria']

df_tax.head()

,Confidence,Kingdom,Phylum,Class,Order,Family,Genus,Species,LowestClassifiedTaxon
ASV,,,,,,,,,
38f9479d2f37d1036274704b09d2e3c5,0.999064,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Blautia,unclassified,Blautia
ffa2d3d4e030a8f45850c379c0d196f5,1.000000,Bacteria,Actinobacteriota,Actinobacteria,Bifidobacteriales,Bifidobacteriaceae,Bifidobacterium,unclassified,Bifidobacterium
f7a1c2a7dc74eb24446faaa79f519ee6,0.991931,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Agathobacter,unclassified,Agathobacter
0ae1308cf1201de65c4edfcd616e89b2,0.715171,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Fusicatenibacter,uncultured-organism,Fusicatenibacter
8a2950244650b024368aa8174145ba33,0.998812,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Blautia,unclassified,Blautia


# Prepare relative abundance table

In [4]:
df_count = pd.read_csv('otu.txt', sep='\t')
df_count = df_count.rename({'#OTU ID':'ASV'}, axis=1).set_index('ASV')
df_count = df_count.div(df_count.sum(axis=0), axis=1)
df_count = df_count[(df_count.T != 0).any()]

# merge tax and absolute abundance
df_count = pd.merge(df_tax[['LowestClassifiedTaxon']], df_count, left_index=True, right_index=True, how='inner').set_index('LowestClassifiedTaxon', drop=True)

# merge index (row sum for the same index)
df_count = df_count.groupby(df_count.index).sum()

# transpose matrix
df_count_T = df_count.T
df_count_T.index.name = 'SampleID'
df_count_T.to_excel('16S_relative_abundance.xlsx')

# show output
df_count_T.head()

LowestClassifiedTaxon,Abiotrophia,Acetanaerobacterium,Acidaminococcus,Actinidia-chinensis,Actinomyces,Actinomyces-massiliensis,Adlercreutzia,Agathobacter,Akkermansia,Akkermansia-muciniphila,...,[Ruminococcus]-torques-group,bacterium-YE57,bacterium-enrichment-culture-clone-BBMC-9,bacterium-enrichment-culture-clone-Ecwsrb026,bacterium-enrichment-culture-clone-RB7c,butyrate-producing-bacterium-L2-10,gut-metagenome,human-gut-metagenome,metagenome,vadinBE97
SampleID,,,,,,,,,,,,,,,,,,,,,
SRR6185161,0.0,0.0,0.004546,0.0,0.000000,0.0,0.0,0.013533,0.002041,0.0,...,0.003009,0.0,0.0,0.000000,0.0,0.0,0.003486,0.000066,0.004507,0.0
SRR6185162,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.001597,0.000000,0.0,...,0.002067,0.0,0.0,0.000000,0.0,0.0,0.001174,0.000869,0.034805,0.0
SRR6185163,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.018110,0.000000,0.0,...,0.007531,0.0,0.0,0.000000,0.0,0.0,0.005917,0.000000,0.026358,0.0
SRR6185164,0.0,0.0,0.000000,0.0,0.000114,0.0,0.0,0.002307,0.000000,0.0,...,0.000773,0.0,0.0,0.000177,0.0,0.0,0.008329,0.000254,0.034066,0.0
SRR6185165,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.052867,0.000000,0.0,...,0.020375,0.0,0.0,0.000220,0.0,0.0,0.002736,0.000000,0.083844,0.0
